https://github.com/osmarluiz/Large-scale-solar-plant-monitoring/blob/main/PV%20Notebook.ipynb

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

import os
import numpy as np
from glob import glob
import glob
import random

from torchvision import transforms
from collections import defaultdict

import segmentation_models_pytorch as smp
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
class GeoImageDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_files = os.listdir(self.img_dir)
        self.mask_files = os.listdir(self.mask_dir)
        self.transform = transform

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        # mask and img_file have so far the same name
        mask_path = os.path.join(self.mask_dir, self.img_files[idx])
        img = torch.load(img_path)
        # converts bool mask into integer (0/1)
        mask = torch.load(mask_path).long()
        # Apply transform (if any)
        if self.transform:
            img = self.transform(img)

        return img, mask

In [18]:
def train(dataloader, model, loss_fn, optimizer, device):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            # loss = loss_fn(pred, y)
            loss = loss_fn(pred.squeeze(1), y.to(torch.float32))

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            return loss


In [14]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # test_loss += loss_fn(pred, y).item()
            test_loss += loss_fn(pred.squeeze(1), y.to(torch.float32))
            loss = loss_fn(pred.squeeze(1), y.to(torch.float32))
            metric = BinaryJaccardIndex().to(device)

            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    jaccard_idx = 100 * metric(pred.squeeze(1), y)
    print(
        f"Test Error: \n"
        f"Jaccard-Index: {(jaccard_idx):>0.3f}%, Avg loss: {test_loss:>5f} \n"
    )
    return loss.item(), jaccard_idx.item()

In [4]:
image_dir = r"C:\Users\Fabian\Documents\Masterarbeit_Daten\images_only_AOI4"
mask_dir = r"C:\Users\Fabian\Documents\Masterarbeit_Daten\masks_only_AOI4"

In [5]:
dataset = GeoImageDataset(image_dir, mask_dir)
train_ds, test_ds = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [6]:
batch_size = 16
shuffle = True

train_dataloader = DataLoader(
    train_ds, batch_size=batch_size, shuffle=shuffle
)

test_dataloader = DataLoader(
    test_ds, batch_size=batch_size, shuffle=shuffle
)

In [20]:
ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['solar_panel']
ACTIVATION = 'sigmoid'
DEVICE = 'cuda'

model = smp.Unet(
    in_channels = 4,
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, #len(CLASSES), 
    activation=ACTIVATION,
    #decoder_atrous_rates = (6, 12, 24) # for DeepLabv3+
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [12]:
loss_fn = smp.losses.DiceLoss(eps=1, mode='binary')
# tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='binary', threshold=0.5)
# metrics = [
#     smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
# ]

optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0005)])

In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, device=DEVICE)
    test(test_dataloader, model, loss_fn, device=DEVICE)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same